## Dependencies

In [1]:
import os
import glob
import torch
from torch.utils.data import ConcatDataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
from src.utils import BrainTumorSegmentationDataset, UNetTester
from pathlib import Path
from src.model1 import UNet
from src.model2 import UNetR2D
from src.model3 import MobileNetV3UNet
from src.train import train_UNet
from src.config.configuration import ConfigurationManager
import gc

## Global parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
H,W = 256,256
torch.manual_seed(42)
batch_size = 16



dataset_path_1 = Path("./data/archive/")
model_path_1 = Path('./models/model1/best_model2.pt')
model_path_2 = Path('./models/model2/best_model3.pt')
model_path_3 = Path('./models/model3/best_model3.pt')
log_path_1 = Path('./logs/test_log_unet_2.csv')
log_path_2 = Path('./logs/test_log_transunet_3.csv')
log_path_3 = Path('./logs/test_log_mobilenetv3_3.csv')


## Data loader

In [2]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((H, W)),
    transforms.ToTensor()
])

test_dataset = BrainTumorSegmentationDataset(
    image_dir= os.path.join(dataset_path_1,'test','images'),
    mask_dir=os.path.join(dataset_path_1,'test','masks'),
    transform=transform
)

test_loader_1 = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

## Test

### Model 1

In [3]:
model_1 = UNet()
output_dir = './logs'
tester = UNetTester(
    model=model_1,
    device=device,
    model_path=model_path_1,
    test_loader=test_loader_1, 
    log_path=log_path_1
)

# 6) Evaluate the model -> compute and save metrics
all_metrics, avg_metrics = tester.evaluate_model()
print(f"Tested on {len(all_metrics)} images.")
print("Average metrics:", avg_metrics)


Per-sample and average metrics saved to: logs\test_log_unet_2.csv
Tested on 16 images.
Average metrics: {'dice': 0.8261965066194534, 'jaccard': 0.7060334607958794, 'precision': 0.847361046820879, 'recall': 0.814082320779562, 'f1': 0.8261915184557438}


### Model 2

In [4]:
configuration = ConfigurationManager()
config = configuration.get_UNetR_params()
model_2 = UNetR2D(config=config)
tester = UNetTester(
    model=model_2,
    device=device,
    model_path=model_path_2,
    test_loader=test_loader_1, 
    log_path=log_path_2
)

# 6) Evaluate the model -> compute and save metrics
all_metrics, avg_metrics = tester.evaluate_model()
print(f"Tested on {len(all_metrics)} images.")
print("Average metrics:", avg_metrics)


Per-sample and average metrics saved to: logs\test_log_transunet_3.csv
Tested on 16 images.
Average metrics: {'dice': 0.6407659780234098, 'jaccard': 0.479664733633399, 'precision': 0.7575067766010761, 'recall': 0.565256156027317, 'f1': 0.6407611425966024}


### Model 3

In [5]:
model_3 = MobileNetV3UNet()
tester = UNetTester(
    model=model_3,
    device=device,
    model_path=model_path_3,
    test_loader=test_loader_1, 
    log_path=log_path_3
)

# 6) Evaluate the model -> compute and save metrics
all_metrics, avg_metrics = tester.evaluate_model()
print(f"Tested on {len(all_metrics)} images.")
print("Average metrics:", avg_metrics)


c:\Projects\python\mainenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Projects\python\mainenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Per-sample and average metrics saved to: logs\test_log_mobilenetv3_3.csv
Tested on 16 images.
Average metrics: {'dice': 0.8603424914181232, 'jaccard': 0.7565244287252426, 'precision': 0.8415287807583809, 'recall': 0.8844862468540668, 'f1': 0.8603375107049942}
